In [1]:
from math import sqrt
import numpy as np
from sklearn.metrics import mean_squared_error, mean_absolute_error, mean_absolute_percentage_error, r2_score
import pandas as pd
from imblearn.under_sampling import RandomUnderSampler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score, precision_score, recall_score

In [2]:
class Node():
    def __init__(self, feature_index=None, threshold=None, left=None, right=None, info_gain=None, value=None):
        self.feature_index = feature_index
        self.threshold = threshold
        self.left = left
        self.right = right
        self.info_gain = info_gain
        self.value = value

In [3]:
class CART():
    def __init__(self, min_samples_split=2, max_depth=2, isRegression = False):
        self.root = None
        self.min_samples_split = min_samples_split
        self.max_depth = max_depth
        self.isRegression = isRegression
        
    def build_tree(self, dataset, curr_depth=0):
        X, Y = dataset[:,:-1], dataset[:,-1]
        num_samples, num_features = np.shape(X)
        if num_samples>=self.min_samples_split and curr_depth<=self.max_depth:
            best_split = self.get_best_split(dataset, num_samples, num_features)
            if best_split["info_gain"]>0:
                left_subtree = self.build_tree(best_split["dataset_left"], curr_depth+1)
                right_subtree = self.build_tree(best_split["dataset_right"], curr_depth+1)
                return Node(best_split["feature_index"], best_split["threshold"], 
                            left_subtree, right_subtree, best_split["info_gain"])

        leaf_value = self.calculate_leaf_value(Y)
        return Node(value=leaf_value)
    
    def get_best_split(self, dataset, num_samples, num_features):
        best_split = {}
        max_info_gain = -float("inf")
        
        for feature_index in range(num_features):
            feature_values = dataset[:, feature_index]
            possible_thresholds = np.unique(feature_values)
            for threshold in possible_thresholds:
                dataset_left, dataset_right = self.split(dataset, feature_index, threshold)
                if len(dataset_left)>0 and len(dataset_right)>0:
                    y, left_y, right_y = dataset[:, -1], dataset_left[:, -1], dataset_right[:, -1]
                    curr_info_gain = self.information_gain(y, left_y, right_y)
                    if curr_info_gain>max_info_gain:
                        best_split["feature_index"] = feature_index
                        best_split["threshold"] = threshold
                        best_split["dataset_left"] = dataset_left
                        best_split["dataset_right"] = dataset_right
                        best_split["info_gain"] = curr_info_gain
                        max_info_gain = curr_info_gain
                        
        return best_split
    
    def split(self, dataset, feature_index, threshold):
        dataset_left = np.array([row for row in dataset if row[feature_index]<=threshold])
        dataset_right = np.array([row for row in dataset if row[feature_index]>threshold])
        return dataset_left, dataset_right
    
    def information_gain(self, parent, l_child, r_child):
        weight_l = len(l_child) / len(parent)
        weight_r = len(r_child) / len(parent)
    
        if self.mode == "entropy":
            gain = self.entropy(parent) - (weight_l*self.entropy(l_child) + weight_r*self.entropy(r_child))
        else:
            gain = self.mean_squared_error(parent) - (weight_l*self.mean_squared_error(l_child) + weight_r*self.mean_squared_error(r_child))
        return gain
    
    def entropy(self, y):
        class_labels = np.unique(y)
        entropy = 0
        for cls in class_labels:
            p_cls = len(y[y == cls]) / len(y)
            entropy += -p_cls * np.log2(p_cls)
        return entropy
    

    
    def mean_squared_error(self, y):
        mean = np.mean(y)
        mse = np.mean((y - mean)**2)
        return mse
        
    def calculate_leaf_value(self, Y):
        if self.isRegression:
            return np.mean(Y)
        Y = list(Y)
        return max(Y, key=Y.count)
    
    def print_tree(self, tree=None, indent=" "):
        if not tree:
            tree = self.root

        if tree.value is not None:
            print(tree.value)

        else:
            print("X_"+str(tree.feature_index), "<=", tree.threshold, "?", tree.info_gain)
            print("%sleft:" % (indent), end="")
            self.print_tree(tree.left, indent + indent)
            print("%sright:" % (indent), end="")
            self.print_tree(tree.right, indent + indent)
    
    def fit(self, X, Y):
        if self.isRegression:
            self.mode = "mse"
        else:
            self.mode = "entropy"
        dataset = np.column_stack((X, Y))
        self.root = self.build_tree(dataset)
    
    def predict(self, X):
        preditions = [self.make_prediction(x, self.root) for x in X]
        return preditions
    
    def make_prediction(self, x, tree):
        if tree.value!=None: return tree.value
        feature_val = x[tree.feature_index]
        if feature_val<=tree.threshold:
            return self.make_prediction(x, tree.left)
        else:
            return self.make_prediction(x, tree.right)

In [8]:
data = pd.read_csv("../data/csgo_task_m.csv")
data.reset_index(inplace=True)
X = data.drop(["bomb_planted"], axis=1).values
y = data["bomb_planted"].values
underSampler = RandomUnderSampler()
X_under_sample, y_under_sample = underSampler.fit_resample(X,y)
X_train, X_test, y_train, y_test = train_test_split(X_under_sample, y_under_sample, test_size=0.2)

In [9]:
tree = CART()
print(y_train.shape)
print(X_train.shape)
tree.fit(X_train, y_train)
predictions = tree.predict(X_test)

(21131,)
(21131, 19)


In [28]:
print(accuracy_score(y_test, predictions))
print(confusion_matrix(y_test, predictions))
print(precision_score(y_test, predictions))
print(recall_score(y_test, predictions))
print(f1_score(y_test, predictions))

1.0
[[2640    0]
 [   0 2643]]
1.0
1.0
1.0


(5283,)

In [12]:
data_regress= pd.read_csv("../data/trip_duration_task_m.csv")
data_regress.drop(["Unnamed: 0"], axis=1, inplace=True)
data_regress = data_regress.head(10000)

In [13]:
y = data_regress['trip_duration'].values
X = data_regress.drop(['trip_duration'], axis=1).values

In [14]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=3)


In [15]:
treeReg = CART(isRegression=True)
treeReg.fit(X_train, y_train)
y_pred = treeReg.predict(X_test)
print(f'MAE: {mean_absolute_error(y_test, y_pred)}')
print(f'MSE: {mean_squared_error(y_test, y_pred)}')
print(f'RMSE: {sqrt(mean_squared_error(y_test, y_pred))}')
print(f'MAPE: {(mean_absolute_percentage_error(y_test, y_pred))}')
print(f'R^2: {r2_score(y_test, y_pred)}')

MAE: 572.4877778260723
MSE: 12731881.111701954
RMSE: 3568.1761604077165
MAPE: 1.1828325376576323
R^2: 0.005660323341293516
